# [API Gateway Micro-project](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/apigateway.html)

-   [LoveToCode: API Gateway](https://www.youtube.com/playlist?list=PLruLATXv4pNz2RPn5X6iMmqyvv69NprJS)
    -   [API Gateway Lambda Mapping (Template Mapping)](https://www.youtube.com/watch?v=o98qYUFSQbc)
    -   [Troubleshooting AWS API Gateway](https://www.youtube.com/watch?v=tUPhFd4Q_Bk)
-   [Complete API Gateway tutorial](https://www.youtube.com/playlist?list=PL5KTLzN85O4ISx9guOjbynlhCxB87xmUq)
-   [AWS API Gateway | REST API for CRUD operations](https://www.youtube.com/watch?v=mQov1eetSec)
-   [Apache Velocity Project](https://velocity.apache.org/)

In [ ]:
import boto3
import botocore
from botocore.exceptions import ClientError
import os, time, json, io, zipfile, subprocess, shutil, requests
from requests_aws4auth import AWS4Auth
from pathlib import Path
from datetime import date
from dotenv import load_dotenv

import s3, iam, lf, glue, lambdafn as lfn, sns, eventbridge as event

from ads.utils import red

In [ ]:
load_dotenv(".env")
VPC_ID              = 'vpc-03617a8a518caa526'
SUBNET_IDS          = ['subnet-0980ad10eb313405b', 'subnet-0de97821ddb8236f7', 'subnet-0a160fbe0fcafe373', 'subnet-0ca765b361e4cb186', 'subnet-0a972b05a5b162feb']
SUBNET_ID           = SUBNET_IDS[0]
SECURITY_GROUP_ID   = 'sg-07f4ccd7a5be677ea'
ACCOUNT_ID          = os.environ['AWS_ACCOUNT_ID_ROOT']
REGION              = os.environ['AWS_DEFAULT_REGION']

In [ ]:
sts_client           = boto3.client('sts')
iam_client           = boto3.client('iam')
s3_client            = boto3.client('s3')
glue_client          = boto3.client('glue')
ec2_client           = boto3.client('ec2', region_name=REGION)
ec2_resource         = boto3.resource('ec2', region_name=REGION)
lfn_client           = boto3.client('lambda')
sfn_client           = boto3.client('stepfunctions')
logs_client          = boto3.client('logs')
events_client        = boto3.client('events')

apigateway_client    = boto3.client('apigateway', region_name=REGION)
sqs_client           = boto3.client('sqs')

-   [Mini Project - Learn to use API Gateway with Lambda, AWS Service and Mock Integrations](https://www.youtube.com/watch?v=sDxsTPbUiik)


#### [How to validate the request in AWS API Gateway](https://www.youtube.com/watch?v=e1x8j8BUEHs)

In [ ]:
api_id = "a1b2c3d4e5"  # Replace with your actual API ID

# Model details
model_name = "MyModel"
description = "Model for user data"
content_type = "application/json"
schema = {
    "type": "object",
    "properties": {
        "id": {"type": "string"},
        "name": {"type": "string"},
        "phone_number": {"type": "string", "pattern": "^[0-9]{10}$"},
        "email_number": {"type": "string", "pattern": "email"}
    },
    "required": ["id", "name"]
}

In [ ]:
apigateway_client.create_model(
    restApiId=api_id,
    name=model_name,
    description=description,
    contentType=content_type,
    schema=json.dumps(schema)  # Convert schema dictionary to JSON string
)

#### [How to enable CloudWatch logs for API Gateway](https://www.youtube.com/watch?v=N49Bp_bd93I&list=PL5KTLzN85O4ISx9guOjbynlhCxB87xmUq&index=16)

In [ ]:
apigateway_client.update_account(
    patchOperations=[
        {
            'op': 'replace',
            'path': '/cloudwatchRoleArn',
            'value': ""  # Use the ARN of the role you created
        }
    ]
)

In [ ]:
LOG_GROUP_NAME = f"/aws/apigateway/{"API_NAME"}"  # Choose a log group name
LOG_LEVEL = 'INFO'  # Options: 'OFF', 'ERROR', 'INFO'


In [ ]:

# Step 1: Create a CloudWatch Log Group
try:
    logs_client.create_log_group(logGroupName=LOG_GROUP_NAME)
except logs_client.exceptions.ResourceAlreadyExistsException:
    print(f"Log group '{LOG_GROUP_NAME}' already exists.")

# Step 2: Enable access logging and method settings for the API Gateway stage
try:
    apigateway_client.update_stage(
        restApiId="API_ID",
        stageName="STAGE_NAME",
        patchOperations=[
            {
                'op': 'replace',
                'path': '/accessLogSettings/destinationArn',
                'value': f'arn:aws:logs:{REGION}:{ACCOUNT_ID}:log-group:{LOG_GROUP_NAME}'  # Update with your REGION and ACCOUNT_ID
            },
            {
                'op': 'replace',
                'path': '/accessLogSettings/format',
                'value': '"$context.identity.sourceIp - $context.requestId"'  # Customize as needed
            },
            {
                'op': 'replace',
                'path': '/*/*/logging/dataTrace',
                'value': 'true'
            },
            {
                'op': 'replace',
                'path': '/*/*/logging/loglevel',
                'value': LOG_LEVEL
            }
        ]
    )

In [ ]:
# Delete the log group:
logs_client.delete_log_group(logGroupName=LOG_GROUP_NAME)

#### [Cross Origin Resource Sharing (CORS)](https://www.youtube.com/watch?v=baQh1X3LN5s)

## REST API

### Technical Notes

#### Integration Pass Through Behavior

The `passthroughBehavior` setting in AWS API Gateway's integration requests controls how incoming requests are handled when API Gateway attempts to map the client request data to the backend. This setting is crucial for managing how API Gateway deals with requests that may not have a perfect match with the expected integration request mapping templates. There are three possible values for `passthroughBehavior`:

1. **WHEN_NO_MATCH**
   - **Description**: The request payload is passed through to the backend **only if** none of the mapping templates specified for the integration matches the `Content-Type` of the incoming request.
   - **Use Case**: This is the most commonly used option as it ensures that, if there's a matching mapping template for the `Content-Type` (like `application/json`), it will be used. If there isn’t, the request will bypass mapping and go straight to the backend.
   - **Example**: If you have a mapping template for `application/json` but receive a request with `Content-Type: text/plain`, API Gateway will pass the request directly to the backend without attempting any mapping transformations.

2. **WHEN_NO_TEMPLATES**
   - **Description**: The request payload is passed through to the backend **if there are no mapping templates defined at all** for the integration request.
   - **Use Case**: This setting is helpful if you want to apply mapping templates selectively based on certain conditions or requests and don’t require strict `Content-Type` validation.
   - **Example**: If no mapping templates are defined for the integration, the request will be sent directly to the backend, regardless of the `Content-Type` in the incoming request.

3. **NEVER**
   - **Description**: The request payload is never passed through to the backend if no matching mapping template is found. Instead, API Gateway will respond with an HTTP `415 Unsupported Media Type` error.
   - **Use Case**: Use this setting if you require strict validation and want to prevent any request from reaching the backend without a defined mapping template.
   - **Example**: If a request has a `Content-Type` not supported by the defined mapping templates, API Gateway will immediately return a `415` error without attempting to forward the request to the backend.

4. **Summary Table**

    | `passthroughBehavior` | Description                                                                                   | Behavior if No Match Found    |
    |-----------------------|-----------------------------------------------------------------------------------------------|-------------------------------|
    | `WHEN_NO_MATCH`       | Pass through if no matching `Content-Type` mapping template is found                         | Forward request to backend    |
    | `WHEN_NO_TEMPLATES`   | Pass through if no mapping templates are defined at all                                      | Forward request to backend    |
    | `NEVER`               | Never pass through if no matching template; return `415 Unsupported Media Type` error instead | Respond with `415` error      |


### [12. AWS API Gateway Input/Output Mapping](https://www.youtube.com/watch?v=HksMXyfFOyM)

-   [13. API Gateway CORS](https://www.youtube.com/watch?v=kDSxCPbBJFo)

### [Creating a Step Functions API using API Gateway](https://docs.aws.amazon.com/step-functions/latest/dg/tutorial-api-gateway.html)

- [YT: Call Your AWS Step Functions With API Gateway](https://www.youtube.com/watch?v=c7mjj-YQXxE&list=PLjfRmoYoxpNoahLvGnz_vJnOZ2FBQhaVu&index=3)

#### Create Role

In [ ]:
API_GATEWAY_ROLE_NAME = "gateway-sfn-role"
LFN_ROLE_NAME = 'lfn-gateway-role'
SFN_ROLE_NAME = 'sfn-pipeline-role'

- **API Gateway Role**

In [ ]:
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "",
            "Effect": "Allow",
            "Principal": {
                "Service": ["apigateway.amazonaws.com"]
            },
            "Action": ["sts:AssumeRole"]
        }
    ]
}

API_GATEWAY_ROLE_ARN =  iam_client.create_role(
    RoleName=API_GATEWAY_ROLE_NAME,
    AssumeRolePolicyDocument=json.dumps(assume_role_policy_document),
    Description='Api Gateway Role'
)['Role']['Arn']

In [ ]:
policy_arns = [
    'arn:aws:iam::aws:policy/AWSStepFunctionsFullAccess', 
    'arn:aws:iam::aws:policy/service-role/AmazonAPIGatewayPushToCloudWatchLogs',
    # 'arn:aws:iam::aws:policy/CloudWatchFullAccess'
]

[iam_client.attach_role_policy(RoleName=API_GATEWAY_ROLE_NAME, PolicyArn=arn) for arn in policy_arns]

In [ ]:
# Policy to enable CloudWatch logging
cloudwatch_logs_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:*"
        }
    ]
}

# # Attach the CloudWatch Logs policy to the role
# response = iam_client.put_role_policy(
#     RoleName=API_GATEWAY_ROLE_NAME,
#     PolicyName='CloudWatchLogsPolicy',
#     PolicyDocument=json.dumps(cloudwatch_logs_policy)
# )

- **Lambda Role**

In [ ]:
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# Create the IAM role with the assume role policy document
LFN_ROLE_ARN = iam_client.create_role(
    RoleName=LFN_ROLE_NAME,
    AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
)['Role']['Arn']

In [ ]:
# Attach the CloudWatch Logs policy to the role
response = iam_client.put_role_policy(
    RoleName=LFN_ROLE_NAME,
    PolicyName='CloudWatchLogsPolicy',
    PolicyDocument=json.dumps(cloudwatch_logs_policy)
)

- **SFN Role**

In [ ]:
stepfunctions_trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "states.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

SFN_ROLE_ARN = iam_client.create_role(
    RoleName=SFN_ROLE_NAME,
    AssumeRolePolicyDocument=json.dumps(stepfunctions_trust_policy),
    Description="Glue Service Role"
)['Role']['Arn']


In [ ]:

# # Attach AWS managed policy with the role
iam_client.attach_role_policy(
    RoleName=SFN_ROLE_NAME,
    PolicyArn='arn:aws:iam::aws:policy/AWSLambda_FullAccess'
)

#### Create Lambda Function

In [ ]:
LFN_APIGATEWAY_NAME = "lfn_in_sfn"

In [ ]:
# lfn.create_lambda_package('./lambdas', "./lambdas/", requirements_file=None)

In [ ]:
# Lambda function code as a string
lambda_code = """
def lambda_handler(event, context):
    print(event)
    return {
        'statusCode': 200,
        'body': 'Hello from Lambda!'
    }
"""

# # Create a ZIP file in memory for the Lambda code
# zip_buffer = io.BytesIO()
# with zipfile.ZipFile(zip_buffer, 'w') as zip_file:
#     zip_file.writestr('lambda_function.py', lambda_code)
# zip_buffer.seek(0)

# Create Lambda function
with open("./lambdas/package.zip", 'rb') as f:
    zipped_code = f.read()


# Create the Lambda function
LFN_ARN = lfn_client.create_function(
    FunctionName=LFN_APIGATEWAY_NAME,
    Runtime='python3.9',  # Specify the runtime version
    Role=LFN_ROLE_ARN,  # Replace with your Lambda execution role ARN
    Handler='hello_world.lambda_handler',
    Code={'ZipFile': zipped_code},
    Description='A simple Lambda function created with boto3',
    Timeout=15,  # Maximum allowed time in seconds for the function to run
    MemorySize=128,  # Allocate memory (128 MB is the minimum)
    Publish=True
)['FunctionArn']


In [ ]:
# Create the CloudWatch log group
response = logs_client.create_log_group(logGroupName=f"/aws/lambda/{LFN_APIGATEWAY_NAME}")

#### SFN

In [ ]:
SMN_NAME = 'api_called_sfn'

In [ ]:
state_machine_definition = {
  "Comment": "A description of my state machine",
  "StartAt": "lfn_gateway",
  "States": {
    "lfn_gateway": {
      "Type": "Task",
      "Resource": "arn:aws:states:::lambda:invoke",
      "OutputPath": "$.Payload",
      "Parameters": {
        "Payload.$": "$",
        "FunctionName": f"arn:aws:lambda:{REGION}:{ACCOUNT_ID}:function:{LFN_APIGATEWAY_NAME}:$LATEST"
      },
      "Retry": [
        {
          "ErrorEquals": [
            "Lambda.ServiceException",
            "Lambda.AWSLambdaException",
            "Lambda.SdkClientException",
            "Lambda.TooManyRequestsException"
          ],
          "IntervalSeconds": 1,
          "MaxAttempts":0,
          "BackoffRate": 2
        }
      ],
      "End": True
    }
  }
}

In [ ]:
SFN_ARN = sfn_client.create_state_machine(
    name=SMN_NAME,
    definition=json.dumps(state_machine_definition),
    roleArn=SFN_ROLE_ARN
)['stateMachineArn']

In [ ]:
response = lfn_client.add_permission(
    FunctionName=LFN_APIGATEWAY_NAME,  # Replace with your Lambda function name
    StatementId=f"{SMN_NAME}_invocation_perm",  # An identifier for this statement, unique for each permission you add
    Action='lambda:InvokeFunction',
    Principal='states.amazonaws.com',
    SourceArn=f"arn:aws:states:{REGION}:{ACCOUNT_ID}:stateMachine:{SMN_NAME}",  # Replace with your S3 bucket ARN
)

In [ ]:
sfn_client.start_execution(stateMachineArn=SFN_ARN, input='{"input": "{}", "name": "MyExecution"}')

#### API Gateway

In [ ]:
API_NAME = 'test_a_post_api'
RESOURCE_NAME = "postdatatosfn"
HTTP_METHOD = 'POST'
STAGE_NAME = 'dev'
DESCRIPTION = 'My REST API Description'
VERSION = '1.0'
ENDPOINT_CONFIGURATION = {'types': ['REGIONAL']}

In [ ]:
params = {
    'name': API_NAME,
    'description': DESCRIPTION,
    'version': VERSION,
    'endpointConfiguration': ENDPOINT_CONFIGURATION,
    # 'apiKeySource': None, # 'HEADER'|'AUTHORIZER',
    # 'policy': 'string',
    # 'tags': None # {'string': 'string'},
    # 'disableExecuteApiEndpoint': True|False
}

In [ ]:
API_ID = apigateway_client.create_rest_api(**params)['id']
# API_ID = response['id']
# print(response)

In [ ]:
# Get the root resource ID
resources = apigateway_client.get_resources(restApiId=API_ID)
print(resources['items']) # Output: [{'id': 'h8kjhm9x8e', 'path': '/'}]

In [ ]:
resource_root_id = next(item['id'] for item in resources['items'] if item['path'] == '/')

In [ ]:
RESOURCE_ID = apigateway_client.create_resource(
    restApiId=API_ID,
    parentId=resource_root_id,
    pathPart=RESOURCE_NAME 
)['id']

In [ ]:
response = apigateway_client.put_method(
    restApiId=API_ID,
    resourceId=RESOURCE_ID,
    httpMethod=HTTP_METHOD,
    authorizationType='NONE',
    apiKeyRequired=False,
    requestParameters= {}
)

In [ ]:
# Step Functions invoke endpoint
apigateway_client.put_integration(
    restApiId=API_ID,
    resourceId=RESOURCE_ID,
    httpMethod='POST',
    type='AWS',
    integrationHttpMethod='POST',
    uri=f'arn:aws:apigateway:{REGION}:states:action/StartExecution',
    credentials=f'arn:aws:iam::{ACCOUNT_ID}:role/{API_GATEWAY_ROLE_NAME}',
    requestTemplates={
        'application/json': f'''{{
            "input": "$util.escapeJavaScript($input.body)",
            "stateMachineArn": "arn:aws:states:{REGION}:{ACCOUNT_ID}:stateMachine:{SMN_NAME}"
        }}'''
    }
)

In [ ]:
# The mapping template to extract the Step Function execution details
mapping_template = '''
#if($input.path('$.executionArn') && $input.path('$.executionArn') != "")
{
  "executionArn": "$input.path('$.executionArn')",
  "startDate": "$input.path('$.startDate')"
}
#else
{
  "errorMessage": "Step function execution failed."
}
#end
'''

# Add the integration response
response = apigateway_client.put_integration_response(
    restApiId=API_ID,
    resourceId=RESOURCE_ID,
    httpMethod=HTTP_METHOD,
    statusCode='200',
    selectionPattern='',
    responseTemplates={
        'application/json': mapping_template
    }
)

print("Integration response configured:", response)

In [ ]:
# Add the method response
response = apigateway_client.put_method_response(
    restApiId=API_ID,
    resourceId=RESOURCE_ID,
    httpMethod=HTTP_METHOD,
    statusCode='200',
    responseModels={
        'application/json': 'Empty'   # Sets the content type for the response
    }
)

In [ ]:
response = apigateway_client.create_deployment(
    restApiId=API_ID,
    stageName=STAGE_NAME
)

In [ ]:
print(f'arn:aws:iam::{ACCOUNT_ID}:role/{API_GATEWAY_ROLE_NAME}')

##### Analysis

In [ ]:
response = apigateway_client.get_integration_response(
    restApiId=API_ID,
    resourceId=RESOURCE_ID,
    httpMethod=HTTP_METHOD,
    statusCode='200'
)

print(response)
response = apigateway_client.get_method_response(
    restApiId=API_ID,
    resourceId=RESOURCE_ID,
    httpMethod=HTTP_METHOD,
    statusCode='200'
)
print(response)

In [ ]:
apigateway_client.get_rest_api(restApiId=API_ID)

In [ ]:
# apigateway_client.get_integration(restApiId=API_ID,resourceId=RESOURCE_ID,httpMethod='POST')

In [ ]:
# apigateway_client.get_method(restApiId=API_ID,resourceId=RESOURCE_ID,httpMethod=HTTP_METHOD)

##### Test API

<b style="color:red">Resources created through boto3 api does not pass test!!</b>

-   AWS CLI client

In [ ]:
! aws apigateway test-invoke-method \
  --rest-api-id {API_ID} \
  --resource-id {RESOURCE_ID} \
  --http-method POST \
  --body '{"input": "{\"name\": \"Momin\", \"age\": 40}", "stateMachineArn": "arn:aws:states:us-east-1:381492255899:stateMachine:apigateway_backend_sfn"}'


- Python Client

In [ ]:
api_url = f"https://{API_ID}.execute-api.{REGION}.amazonaws.com" # 
post_endpoint = f"{api_url}/{STAGE_NAME}/{RESOURCE_NAME}"; post_endpoint
print()

In [ ]:

# # AWS credentials
# session = boto3.Session()
# credentials = session.get_credentials()
# auth = AWS4Auth(credentials.access_key, credentials.secret_key, REGION, 'execute-api', session_token=credentials.token)

# Define headers
headers = {
    'Content-Type': 'application/json',
    # Uncomment if using API key
    # 'x-api-key': 'your-api-key',
}

# Define the payload to trigger the Step Function
payload = {
   "input": json.dumps({
       "name": "Momin",
       "age": 40
   }),
   "stateMachineArn": f"arn:aws:states:{REGION}:{ACCOUNT_ID}:stateMachine:{SMN_NAME}"
}

# Send the POST request with SigV4 signing
response = requests.post(post_endpoint, headers=headers, data=json.dumps(payload))

# # Check the response status and data
# if response.status_code == 200:
#     print("POST request was successful!")
#     print("Response data:", response.json())
# else:
#     print(f"Failed with status code: {response.status_code}")
#     print("Response:", response.text)

print(response)

In [ ]:
print(f"arn:aws:states:{REGION}:{ACCOUNT_ID}:stateMachine:{SMN_NAME}")
# print(post_endpoint)
{
   "input": "{\"name\": \"Momin\", \"age\": 40}",
   "stateMachineArn": "arn:aws:states:us-east-1:381492255899:stateMachine:apigateway_backend_sfn"
}

-   Test the API using `curl` command

In [ ]:
!curl -X POST "https://kz8tizmso1.execute-api.us-east-1.amazonaws.com/dev/postdatatosfn" \
-H "Content-Type: application/json" \
-d '{"input": "{\"name\": \"Momin\", \"age\": 40}", "stateMachineArn": "arn:aws:states:us-east-1:381492255899:stateMachine:apigateway_backend_sfn"}'


#### Delete Resources

In [ ]:
lfn_client.delete_function(FunctionName=LFN_APIGATEWAY_NAME)

In [ ]:
response = apigateway_client.delete_rest_api(restApiId=API_ID)

In [ ]:
sfn_client.delete_state_machine(stateMachineArn=SFN_ARN)

In [ ]:
## DELETE IAM ROLE AT THE END AFTER DELETING ALL OTHER RESOURCES.
iam.delete_iam_role(API_GATEWAY_ROLE_NAME)
iam.delete_iam_role(LFN_ROLE_NAME)
iam.delete_iam_role(SFN_ROLE_NAME)

### SQS integration with API Gateway


-   [part-1](https://www.youtube.com/watch?v=v8cqDXyHZ4Y&list=PL5KTLzN85O4ISx9guOjbynlhCxB87xmUq&index=21) || `SUCCESS`
-   [part-2: Send message to SQS using query string not through Body](https://www.youtube.com/watch?v=eoVv455UEJE&list=PL5KTLzN85O4ISx9guOjbynlhCxB87xmUq&index=22) || `SUCCESS`
-   [part-3: Fetch message from SQS](https://www.youtube.com/watch?v=1iZpzCu3ZmE&list=PL5KTLzN85O4ISx9guOjbynlhCxB87xmUq&index=23) || `SUCCESS`

#### SQS

In [ ]:
QUE_NAME = 'lambda_integrated_sqs'
attributes = {
    'DelaySeconds': '0',
    'MaximumMessageSize': '262144',  # 256 KB
    'MessageRetentionPeriod': '345600',  # 4 days
    'ReceiveMessageWaitTimeSeconds': '0',
    'VisibilityTimeout': '30', # 30 secoonds
}

In [ ]:
# Create the SQS queue with the specified attributes
que_url = sqs_client.create_queue(
    QueueName=QUE_NAME,
    Attributes=attributes
)

In [ ]:
# que_url = sqs_client.get_queue_url(QueueName=QUE_NAME)['QueueUrl']

In [ ]:
# message_body = 'This is a SQS test message'
# message_attributes = {}
# response = sqs_client.send_message(
#             QueueUrl=que_url,
#             MessageBody=message_body,
#             MessageAttributes=message_attributes or {}
#         )
# message_id = response['MessageId']

In [ ]:
response = sqs_client.receive_message(
            QueueUrl=que_url,
            MaxNumberOfMessages=10,
            WaitTimeSeconds=10
        )
messages = response.get('Messages', [])

len(messages)

In [ ]:
# Delete messages
for message in messages:
    print(message['Body'])
    # sqs_client.delete_message(
    #     QueueUrl=que_url,
    #     ReceiptHandle=message['ReceiptHandle']
    # )

#### IAM Role

In [ ]:
API_GATEWAY_ROLE_NAME = "apigateway_sqs_role"

In [ ]:
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "",
            "Effect": "Allow",
            "Principal": {
                "Service": ["apigateway.amazonaws.com"]
            },
            "Action": ["sts:AssumeRole"]
        }
    ]
}

API_GATEWAY_ROLE_ARN =  iam_client.create_role(
    RoleName=API_GATEWAY_ROLE_NAME,
    AssumeRolePolicyDocument=json.dumps(assume_role_policy_document),
    Description='Api Gateway Role'
)['Role']['Arn']

In [ ]:
policy_arns = [
    'arn:aws:iam::aws:policy/service-role/AmazonAPIGatewayPushToCloudWatchLogs',
    # 'arn:aws:iam::aws:policy/CloudWatchFullAccess'
]

[iam_client.attach_role_policy(RoleName=API_GATEWAY_ROLE_NAME, PolicyArn=arn) for arn in policy_arns]

In [ ]:
sqs_send_message_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "",
            "Effect": "Allow",
            "Action": [
                "sqs:SendMessage",
                "sqs:ReceiveMessage",
                "sqs:GetQueueAttributes"
            ],
            "Resource": f"arn:aws:sqs:{REGION}:{ACCOUNT_ID}:{QUE_NAME}",
        }
    ]
}

iam_client.put_role_policy(
    RoleName=API_GATEWAY_ROLE_NAME,
    PolicyName="sqs_send_Receive_message_from_apigateway",
    PolicyDocument=json.dumps(sqs_send_message_policy_document)
)

#### API Gateway

In [ ]:
API_NAME = 'sqs_api_endpoint'
RESOURCE_NAME = "sqs"
HTTP_METHOD = 'POST'
STAGE_NAME = 'dev'
DESCRIPTION = 'A SQS intergrated api gateway endpoint'
VERSION = '1.0'
ENDPOINT_CONFIGURATION = {'types': ['REGIONAL']}

In [ ]:
params = {
    'name': API_NAME,
    'description': DESCRIPTION,
    'version': VERSION,
    'endpointConfiguration': ENDPOINT_CONFIGURATION,
    'disableExecuteApiEndpoint': False,
    'apiKeySource': 'HEADER' #|'AUTHORIZER',
    # 'policy': 'string',
    # 'tags': None # {'string': 'string'},
}

In [ ]:
API_ID = apigateway_client.create_rest_api(**params)['id']
# print(API_ID)

In [ ]:
# Get the root resource ID
resources = apigateway_client.get_resources(restApiId=API_ID)
print(resources['items']) # Output: [{'id': 'h8kjhm9x8e', 'path': '/'}]

In [ ]:
resource_root_id = next(item['id'] for item in resources['items'] if item['path'] == '/')

In [ ]:
RESOURCE_ID = apigateway_client.create_resource(
    restApiId=API_ID,
    parentId=resource_root_id,
    pathPart=RESOURCE_NAME 
)['id']

##### Part 1: Send Message a `Message-Body` with `HTTP Headers`

In [ ]:
response = apigateway_client.put_method(
    restApiId=API_ID,
    resourceId=RESOURCE_ID,
    httpMethod=HTTP_METHOD,
    authorizationType='NONE',
    apiKeyRequired=False,
    requestParameters= {}
)

In [ ]:
apigateway_client.put_integration(
    restApiId=API_ID,
    resourceId=RESOURCE_ID,
    httpMethod='POST',
    type='AWS',
    integrationHttpMethod='POST',
    uri=f'arn:aws:apigateway:{REGION}:sqs:path/{ACCOUNT_ID}/{QUE_NAME}', ## Only for Action-Type = "Use path override"
    # uri=f'arn:aws:apigateway:{REGION}:sqs:action/SendMessage',         ## Only for Action-Type = "Use action Name"
    credentials=f'arn:aws:iam::{ACCOUNT_ID}:role/{API_GATEWAY_ROLE_NAME}',

    requestParameters={
        'integration.request.header.Content-Type': "'application/x-www-form-urlencoded'", ## set value for 'HTTP Headers'
        # Since you are specifying the queue ARN directly in the integration uri, you don't need to add QueueUrl as a parameter.
        # 'integration.request.querystring.QueueUrl': f"'{que_url}'"  # Set QueueUrl
    },
    requestTemplates={
        'application/json': "Action=SendMessage&MessageBody=$input.body" # set value for 'Mapping Templates'
    },
    passthroughBehavior='NEVER',
)


In [ ]:
# response = apigateway_client.delete_integration(
#     restApiId=API_ID,
#     resourceId=RESOURCE_ID,
#     httpMethod=HTTP_METHOD
# )

In [ ]:
mapping_template = """
#set($inputRoot = $input.path('$'))
{
    "MessageId": "$inputRoot.SendMessageResponse.SendMessageResult.MessageId",
    "MD5OfMessageBody": "$inputRoot.SendMessageResponse.SendMessageResult.MD5OfMessageBody"
}
"""

# Add the integration response
apigateway_client.put_integration_response(
    restApiId=API_ID,
    resourceId=RESOURCE_ID,
    httpMethod=HTTP_METHOD,
    statusCode='200',
    selectionPattern='',
    responseTemplates={
        'application/json': mapping_template
    }
)

# print("Integration response configured:", response)

In [ ]:
# response = apigateway_client.delete_integration_response(
#     restApiId=API_ID,
#     resourceId=RESOURCE_ID,
#     httpMethod=HTTP_METHOD,
#     statusCode='200',
# )

In [ ]:
# Add the method response
response = apigateway_client.put_method_response(
    restApiId=API_ID,
    resourceId=RESOURCE_ID,
    httpMethod=HTTP_METHOD,
    statusCode='200',
    responseModels={
        'application/json': 'Empty'   # Sets the content type for the response
    }
)

In [ ]:
response = apigateway_client.create_deployment(
    restApiId=API_ID,
    stageName=STAGE_NAME
)

In [ ]:
print(f'arn:aws:iam::{ACCOUNT_ID}:role/{API_GATEWAY_ROLE_NAME}')

##### Part 2: Send Message as `Query String Parameter`

In [ ]:
API_ID="h2jhdnm2gl"
RESOURCE_ID="9yiik7"

In [ ]:
apigateway_client.put_method(
    restApiId=API_ID,
    resourceId=RESOURCE_ID,
    httpMethod=HTTP_METHOD,
    authorizationType='NONE',
    apiKeyRequired=False,
    requestParameters={
        "method.request.querystring.MessageBody": True  # method.request.querystring.<parameter-name>: Replace <parameter-name> with the actual query parameter you expect. The value (True or False) indicates whether the parameter is required.
    }
)

In [ ]:
# apigateway_client.delete_method(
#     restApiId=API_ID,
#     resourceId=RESOURCE_ID,
#     httpMethod=HTTP_METHOD
# )

In [ ]:
apigateway_client.put_integration(
    restApiId=API_ID,
    resourceId=RESOURCE_ID,
    httpMethod='POST',
    type='AWS',
    integrationHttpMethod='POST',
    uri=f'arn:aws:apigateway:{REGION}:sqs:path/{ACCOUNT_ID}/{QUE_NAME}',   # For path override
    # uri=f'arn:aws:apigateway:{REGION}:sqs:action/SendMessage',           # For action name
    
    credentials=f'arn:aws:iam::{ACCOUNT_ID}:role/{API_GATEWAY_ROLE_NAME}',

    requestParameters={
        "integration.request.querystring.Action": "'SendMessage'",  # Literal value
        "integration.request.querystring.MessageBody": "method.request.querystring.MessageBody"
    },
    
    requestTemplates={
        "application/x-www-form-urlencoded": (
            "Action=SendMessage&"
            "MessageBody=$input.params('MessageBody')"
        )
    },

    passthroughBehavior='WHEN_NO_MATCH',  # Use WHEN_NO_MATCH to avoid 415 error if template doesn't match
)

In [ ]:
# apigateway_client.delete_integration(
#     restApiId=API_ID,
#     resourceId=RESOURCE_ID,
#     httpMethod=HTTP_METHOD
# )

In [ ]:
# Add Method Response for HTTP 200 status code
apigateway_client.put_method_response(
    restApiId=API_ID,
    resourceId=RESOURCE_ID,
    httpMethod='POST',
    statusCode='200',
    responseParameters={
        "method.response.header.Content-Type": False
    },
    responseModels={
        "application/json": "Empty"
    }
)

# Add Integration Response to handle SQS's HTTP 200 response
apigateway_client.put_integration_response(
    restApiId=API_ID,
    resourceId=RESOURCE_ID,
    httpMethod='POST',
    statusCode='200',
    selectionPattern='',
    responseParameters={
        "method.response.header.Content-Type": "integration.response.header.Content-Type"
    },
    responseTemplates={
        "application/json": ""
    }
)


In [ ]:
# response = apigateway_client.delete_integration_response(
#     restApiId=API_ID,
#     resourceId=RESOURCE_ID,
#     httpMethod=HTTP_METHOD,
#     statusCode='200',
# )

In [ ]:
response = apigateway_client.create_deployment(
    restApiId=API_ID,
    stageName=STAGE_NAME
)

##### Part 3: Fetch Message from SQS

In [ ]:
# API_ID="83y4nly4ag"
# RESOURCE_ID="dkpp7wkarg"
HTTP_METHOD = "GET"
INTEGRATION_HTTP_METHOD = "GET"

In [ ]:
apigateway_client.put_method(
    restApiId=API_ID,
    resourceId=RESOURCE_ID,
    httpMethod="GET",
    authorizationType='NONE',
    apiKeyRequired=False,
    requestParameters={
        "method.request.querystring.MessageBody": True  # method.request.querystring.<parameter-name>: Replace <parameter-name> with the actual query parameter you expect. The value (True or False) indicates whether the parameter is required.
    }
)

In [ ]:
# apigateway_client.delete_method(
#     restApiId=API_ID,
#     resourceId=RESOURCE_ID,
#     httpMethod=GET_HTTP_METHOD
# )

In [ ]:
apigateway_client.put_integration(
    restApiId=API_ID,
    resourceId=RESOURCE_ID,
    httpMethod="GET",
    type='AWS',
    integrationHttpMethod="GET",
    uri=f'arn:aws:apigateway:{REGION}:sqs:path/{ACCOUNT_ID}/{QUE_NAME}',   # For path override
    # uri=f'arn:aws:apigateway:{REGION}:sqs:action/SendMessage',               # For action name
    
    credentials=f'arn:aws:iam::{ACCOUNT_ID}:role/{API_GATEWAY_ROLE_NAME}',

    requestParameters={
        "integration.request.querystring.Action": "'ReceiveMessage'",  # Literal value
        # "integration.request.querystring.MessageBody": "method.request.querystring.MessageBody"
    },
    passthroughBehavior='WHEN_NO_MATCH',  # Use WHEN_NO_MATCH to avoid 415 error if template doesn't match
)

In [ ]:
# apigateway_client.delete_integration(
#     restApiId=API_ID,
#     resourceId=RESOURCE_ID,
#     httpMethod="GET"
# )

In [ ]:
# Add Method Response for HTTP 200 status code
apigateway_client.put_method_response(
    restApiId=API_ID,
    resourceId=RESOURCE_ID,
    httpMethod="GET",
    statusCode='200',
    responseParameters={
        "method.response.header.Content-Type": False  # Not required, but can include
    },
    responseModels={
        "application/json": "Empty"  # No specific model required; pass through SQS response
    }
)


In [ ]:
# Add Integration Response to handle SQS's HTTP 200 response
apigateway_client.put_integration_response(
    restApiId=API_ID,
    resourceId=RESOURCE_ID,
    httpMethod="GET",
    statusCode='200',
    selectionPattern='',
    responseParameters={
        "method.response.header.Content-Type": "integration.response.header.Content-Type"
    },
    responseTemplates={
        # Define a response template to format the SQS response as JSON
        "application/json": """
            #set($inputRoot = $input.path('$'))
            {
                "Messages": $inputRoot.Messages
            }
        """
    }
)


In [ ]:
# response = apigateway_client.delete_integration_response(
#     restApiId=API_ID,
#     resourceId=RESOURCE_ID,
#     httpMethod=HTTP_METHOD,
#     statusCode='200',
# )

In [ ]:
apigateway_client.create_deployment(
    restApiId=API_ID,
    stageName=STAGE_NAME
)

##### Analysis

In [ ]:
apigateway_client.get_integration_response(
    restApiId=API_ID,
    resourceId=RESOURCE_ID,
    httpMethod=HTTP_METHOD,
    statusCode='200'
)

In [ ]:
apigateway_client.get_method_response(
    restApiId=API_ID,
    resourceId=RESOURCE_ID,
    httpMethod=HTTP_METHOD,
    statusCode='200'
)

In [ ]:
apigateway_client.get_rest_api(restApiId=API_ID)

In [ ]:
apigateway_client.get_integration(restApiId=API_ID,resourceId=RESOURCE_ID,httpMethod='POST')

In [ ]:
# apigateway_client.get_method(restApiId=API_ID,resourceId=RESOURCE_ID,httpMethod=HTTP_METHOD)

##### Test API

In [ ]:
api_url = f"https://{API_ID}.execute-api.{REGION}.amazonaws.com" # 
API_ENDPOINT_URL = f"{api_url}/{STAGE_NAME}/{RESOURCE_NAME}"
print(API_ENDPOINT_URL)

###### Part 1

In [ ]:
import requests
import json

# Replace this with your actual API Gateway endpoint
# API_ENDPOINT_URL = "https://<api_id>.execute-api.<region>.amazonaws.com/<stage>/<resource>"

# Sample data to send in the request body
data = {
    "input": json.dumps({
        "name": "Momin",
        "age": 40
    })
}

# Headers (add an API key or other authentication if needed)
headers = {
    "Content-Type": "application/json"
}

try:
    # Sending a POST request
    response = requests.post(API_ENDPOINT_URL, headers=headers, data=json.dumps(data))

    # Check if the request was successful
    if response.status_code == 200:
        print("Request succeeded:", response.json())
    else:
        print("Request failed with status:", response.status_code)
        print("Response:", response.text)

except requests.exceptions.RequestException as e:
    print("An error occurred:", e)


###### Part 2:

In [ ]:
import requests

# Replace these values with your actual API details
api_url = "https://<api-id>.execute-api.<region>.amazonaws.com/<stage>/send-message"
message_body = "Hello, this is a test message for SQS via API Gateway!"

# Send a POST request with the query string parameter `MessageBody`
response = requests.post(API_ENDPOINT_URL,params={"MessageBody": message_body})

# Output the response
print(f"Status Code: {response.status_code}")
print("Response Body:", response.json() if response.headers["Content-Type"] == "application/json" else response.text)


###### Part 3:

In [ ]:
# Define the query parameters, if required
params = {
    "MessageBody": "SampleMessage"  # Include this only if your API expects a query parameter
}

# Send the GET request
response = requests.get(API_ENDPOINT_URL, params=params)
print(response)

# # Output the response
# print(f"Status Code: {response.status_code}")
print("Response Headers:", response.headers)
# print("Response Body:", response.json() if response.headers.get("Content-Type") == "application/json" else response.text)

#### Delete Resources

In [ ]:
sqs_client.delete_queue(QueueUrl=que_url)

In [ ]:
apigateway_client.delete_rest_api(restApiId=API_ID)

In [ ]:
# Remove all inline policies
inline_policies = iam_client.list_role_policies(RoleName=API_GATEWAY_ROLE_NAME)['PolicyNames']
for policy_name in inline_policies:
    iam_client.delete_role_policy(RoleName=API_GATEWAY_ROLE_NAME, PolicyName=policy_name)
    print(f"Deleted inline policy {policy_name} from role {API_GATEWAY_ROLE_NAME}")

In [ ]:
iam.delete_iam_role(API_GATEWAY_ROLE_NAME)


### [AWS Tutorials - Deploy Simple Web Applications in Lambda](https://www.youtube.com/watch?v=a6ltARyR9J8)

-   [lab](https://aws-dojo.com/ws47/labs/)

<div style="text-align:center"><img src="./web_app_with_lfn/images/api_lfn_webapp1.png" height="260p" height="200p"></img</dev>
<div style="text-align:center"><img src="./web_app_with_lfn/images/api_lfn_webapp2.png" height="260p" height="200p"></img</dev>

- Only Prototype has been created

#### Create IAM Role

In [ ]:
LFN_ROLE_NAME = 'lfn-pipeline-role'

In [ ]:
policy_arns = [
    # "arn:aws:iam::aws:policy/service-role/AWSGlueServiceRole",
    # "arn:aws:iam::aws:policy/CloudWatchFullAccess",
    # "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    # "arn:aws:iam::aws:policy/AdministratorAccess",
    "arn:aws:iam::aws:policy/PowerUserAccess"
]

In [ ]:
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# Create the IAM role with the assume role policy document
LFN_ROLE_ARN = iam_client.create_role(
    RoleName=LFN_ROLE_NAME,
    AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
)['Role']['Arn']

In [ ]:
[iam_client.attach_role_policy(RoleName=LFN_ROLE_NAME, PolicyArn=parn) for parn in policy_arns]

#### Create DynamoDB Table

In [ ]:
dynamodb_client.create_table(
            TableName=table_name,
            KeySchema=key_schema,
            AttributeDefinitions=attribute_definitions,
            ProvisionedThroughput={
                'ReadCapacityUnits': read_capacity_units,
                'WriteCapacityUnits': write_capacity_units
            }
        )

#### Create Lambda Function

In [ ]:
LFN_NAME = "httx-web-handler"
zip_file = "./lambdas/lfn1/package.zip"  # Change this to the actual zip file path

# Create Lambda function
with open(zip_file, 'rb') as f:
    zipped_code = f.read()

LFN_CRAWLER_ARN = lambda_client.create_function(
    FunctionName=LFN_NAME,
    Runtime='python3.8',
    Role=LFN_ROLE_ARN,
    Handler='httx-web-handler.lambda_handler',
    Code={'ZipFile': zipped_code},
    Timeout=120,
    Environment={
        'Variables': {
            'foo': 'BAR'
        }
    }
)['FunctionArn']

#### API Gateway

In [ ]:
API_NAME = 'sqs_api_endpoint'
RESOURCE_NAME = "sqs"
HTTP_METHOD = 'POST'
STAGE_NAME = 'dev'
DESCRIPTION = 'A SQS intergrated api gateway endpoint'
VERSION = '1.0'
ENDPOINT_CONFIGURATION = {'types': ['REGIONAL']}

In [ ]:
params = {
    'name': API_NAME,
    'description': DESCRIPTION,
    'version': VERSION,
    'endpointConfiguration': ENDPOINT_CONFIGURATION,
    'disableExecuteApiEndpoint': False,
    'apiKeySource': 'HEADER' #|'AUTHORIZER',
    # 'policy': 'string',
    # 'tags': None # {'string': 'string'},
}

In [ ]:
API_ID = apigateway_client.create_rest_api(**params)['id']
# print(API_ID)

In [ ]:
# Get the root resource ID
resources = apigateway_client.get_resources(restApiId=API_ID)
print(resources['items']) # Output: [{'id': 'h8kjhm9x8e', 'path': '/'}]

In [ ]:
resource_root_id = next(item['id'] for item in resources['items'] if item['path'] == '/')

In [ ]:
RESOURCE_ID = apigateway_client.create_resource(
    restApiId=API_ID,
    parentId=resource_root_id,
    pathPart=RESOURCE_NAME 
)['id']